In [5]:
import torch
import torchmetrics
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedGroupKFold
from torch.utils.data import DataLoader, Dataset, Subset

In [6]:
'''
currently available samples:
SQ1631_s1_R, SQ1631_s2_R, SQ1631_s3_N, SQ1631_s4_N
SQ1632_s1_R, SQ1632_s2_N, SQ1632_s3_R, SQ1632_s4_N
SQ1633_s1_R, SQ1633_s2_R, SQ1633_s3_N, SQ1633_s4_N
'''
sample_ls = ['SQ1631_s1_R', 'SQ1631_s2_R', 'SQ1631_s3_N', 'SQ1631_s4_N',
             'SQ1632_s1_R', 'SQ1632_s2_N', 'SQ1632_s3_R', 'SQ1632_s4_N',
             'SQ1633_s1_R', 'SQ1633_s2_R', 'SQ1633_s3_N', 'SQ1633_s4_N']


In [ ]:
# get all channel means
channel_names = []
channel_means = []
for sample_name in sample_ls:
    slide = np.load(f'../data/npy/{sample_name}.npy', mmap_mode='r')
    mask = np.sum(slide, axis=0) > 0
    nonzero = slide[:, mask]
    channel_mean = np.mean(nonzero, axis=1)
    channels = pd.read_csv(f'../data/npy/{sample_name}_channel_names.csv', header=None, index_col=0)
    channels = [name[0] for name in channels.to_numpy()]
    channels = [mz.split('-±')[0] for mz in channels]
    channel_means.append(pd.DataFrame(channel_mean, index=channels, columns=[sample_name]))
    channel_names.append(channels)
channel_means = pd.concat(channel_means, axis=1)

In [ ]:
# check if all channels are in the same order
def all_arrays_identical(array_list):
    if not array_list:
        return True  # An empty list is considered to have all identical elements
    first_array = array_list[0]
    return all(np.array_equal(first_array, arr) for arr in array_list)

all_arrays_identical(channel_names)

In [ ]:
channel_means.head()

In [ ]:
sns.clustermap(channel_means.T, cmap="Reds", vmin=0, vmax=255, figsize=(10, 4))
plt.show()

In [ ]:
# try random forest classifier on channel_means data
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier

In [ ]:
labels = [name[-1] for name in channel_means.columns]
print(labels)
response = [1 if lab == 'R' else 0 for lab in labels]
print(response)

X = channel_means.T.to_numpy()
y = np.asarray(response)

# Create a RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=0)

# Create a KFold object with 4 splits
kf = KFold(n_splits=4, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(rf, X, y, cv=kf)

# Print the cross-validation scores
print("Cross-validation scores for each fold: ", cv_scores)
print("Mean cross-validation score: ", np.mean(cv_scores))

In [ ]:
# Initialize lists to store predictions and true labels
all_predictions = []
all_true_labels = []

# Perform manual cross-validation

for train_index, test_index in kf.split(X):
    # Split the data into training and validation sets
    print(train_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the model on the training set
    rf.fit(X_train, y_train)
    
    # Predict on the validation set
    y_pred = rf.predict(X_test)
    
    # Store predictions and true labels
    all_predictions.append(y_pred)
    all_true_labels.append(y_test)
    
    # Print predictions and true labels for the current fold
    print(f"Predictions for fold: {y_pred}")
    print(f"True labels for fold: {y_test}")
    print("----------")

# Convert the lists of arrays to a single array
all_predictions = np.concatenate(all_predictions)
all_true_labels = np.concatenate(all_true_labels)

# Print overall predictions and true labels
print("All predictions:", all_predictions)
print("All true labels:", all_true_labels)

In [ ]:
# inspect individual samples
sample_name = sample_ls[0]
slide = np.load(f'../data/npy/{sample_name}.npy', mmap_mode='r')
channels = pd.read_csv(f'../data/npy/{sample_name}_channel_names.csv', header=None, index_col=0)
channels = [name[0] for name in channels.to_numpy()]
print(channels[0:10])
channels = [mz.split('-±')[0] for mz in channels]
print(channels[0:10])

In [ ]:
print(slide.shape)
mask = np.sum(slide, axis=0) > 0

In [ ]:
plt.imshow(mask, cmap='gray')

In [ ]:
plt.imshow(slide[750, ], cmap='gray')

In [ ]:
# channel wise correlation
nonzero = slide[:, mask]
print(f'None zero pixels: {nonzero.shape}')
corr = np.corrcoef(nonzero)

In [ ]:
sns.clustermap(pd.DataFrame(corr, index=channels, columns=channels), cmap="RdBu_r", vmin=-1, vmax=1, figsize=(6, 6))
plt.show()
#plt.imshow(corr, cmap='RdBu')

In [ ]:
# create tiles
def tile_array(array, tile_size, overlap, threshold):
    c, h, w = array.shape
    step_size = tile_size - overlap
    nx = w // step_size + 1
    ny = h // step_size + 1
    mask = np.sum(array, axis=0) > 0
    tiles = []
    coords = []
    for i in range(nx):
        for j in range(ny):
            
            x = i * step_size
            y = j * step_size
            x_end = min((x + tile_size), w)
            y_end = min((y + tile_size), h)
            if np.mean(mask[y :y_end, x : x_end]) > threshold:
                tile = np.zeros((c, tile_size, tile_size))
                tile[:,: y_end-y, :x_end-x] = array[:, y :y_end, x : x_end]
                tiles.append(tile)
                coords.append([x, y])
            else:
                pass
    return tiles, coords

class TileDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
        from torchvision.transforms import v2
        self.transform = v2.Compose([
                                        #v2.ToImage(),
                                        v2.ToDtype(torch.float, scale=True),
                                        #v2.Lambda(lambda x: (x - 0.5)*2)
                                        ])
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        #x = np.expand_dims(self.inputs[index], 0)
        x = torch.Tensor(self.inputs[index])
        x = self.transform(x)
        #print(x.shape)
        y = torch.tensor(self.labels[index], dtype=torch.float)
        return x, y

    
class SimpleCNN(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_channels, 16, 1),
                                   nn.ReLU(),
                                   nn.Conv2d(16, 8, 3, stride=2),
                                   nn.ReLU(),
                                   nn.Conv2d(8, 8, 3, stride=2),
                                   nn.ReLU(),
                                   nn.Conv2d(8, 8, 3, stride=2),
                                   nn.ReLU()
                                  )
        self.fc = nn.Linear(8, 1)

    def forward(self, x):
        x = self.conv(x)
        #print(x.shape)
        x = torch.mean(x, dim = (2, 3), keepdim = False) # Global pooling
        #print(x.shape)
        out = self.fc(x)
        out = torch.squeeze(out)
        if out.ndim == 0:
            out = out.unsqueeze(0)
        return out
    
def train_and_evaluate(n_epochs, model, trn_loader, tst_loader):
    criterion = nn.BCEWithLogitsLoss()
    accuracy_fun = torchmetrics.Accuracy(task='binary')
    roc_fun = torchmetrics.AUROC(task='binary')
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(n_epochs):
        print('EPOCH {}:'.format(epoch + 1))
        model.train()
        running_loss = 0.
        for i, (inputs, labels) in enumerate(trn_loader):
            optimizer.zero_grad()
            logits = model(inputs)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 10 == 9:  # Print every 100 batches
                print(f'Epoch [{epoch + 1}/{n_epochs}], Step [{i + 1}/{len(trn_loader)}], Loss: {running_loss / 100:.4f}')
                running_loss = 0.0
    print('Finished Training')

    # test model and report metric
    model.eval()
    running_loss = 0.
    all_probs = []
    all_labels = []
    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(tst_loader):
            logits = model(inputs)
            all_labels.append(labels)
            all_probs.append(nn.Sigmoid()(logits).cpu())
            loss = criterion(logits, labels)
            running_loss += loss
    all_probs = torch.cat(all_probs)
    print(all_probs)
    all_labels = torch.cat(all_labels)
    print(all_labels)
    avg_loss = running_loss / (i + 1)
    accuracy = accuracy_fun(all_probs, all_labels)
    auc = roc_fun(all_probs, all_labels)
    print(f'Test loss: {avg_loss}; AUROC: {auc}; Accuracy: {accuracy}')
    return accuracy, auc
    

In [ ]:
tile_ls = []
label_ls = []
tile_grp = []
tile_coords = []

for sample_name in sample_ls[0:4]:
    slide = np.load(f'../data/npy/{sample_name}.npy', mmap_mode='r')
    tiles, coords = tile_array(slide, tile_size=32, overlap=8, threshold=0.2)
    labels = [1 if sample_name[-1] == 'R' else 0] * len(tiles)
    print(f'{len(tiles)} tiles generated in sample {sample_name}.')
    tile_ls.extend(tiles)
    label_ls.extend(labels)
    tile_grp.extend([sample_name] * len(tiles))
    tile_coords.extend(coords)

tile_ls = np.asarray(tile_ls)
label_ls = np.asarray(label_ls)
print(tile_ls.shape)
print(label_ls.shape)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_epochs = 10 
print(tile_ls[0].shape)
tile_dat = TileDataset(inputs=tile_ls, labels=label_ls)
print(len(tile_dat))

def collate_fn(batch):
    inputs, labels = zip(*batch)
    #print(inputs[0].shape)
    inputs = torch.stack(inputs, 0)
    labels = torch.stack(labels)
    #print(inputs.shape)
    #print(labels)
    inputs = inputs.to(device)
    labels = labels.to(device)
    return inputs, labels
        
sgkf = StratifiedGroupKFold(n_splits=4, shuffle=True, random_state=42)

for i, (trn_idx, tst_idx) in enumerate(sgkf.split(X=np.zeros(len(label_ls)), y=label_ls, groups=tile_grp)):
    print(f'Fold {i}')
    print('--------------------------------')
    trn_subset = Subset(tile_dat, trn_idx)
    tst_subset = Subset(tile_dat, tst_idx)
    print(trn_idx)
    print(tst_idx)
    # Define data loaders for training and testing data in this fold
    trn_loader = DataLoader(trn_subset, batch_size=4, shuffle=True, collate_fn=collate_fn)
    tst_loader = DataLoader(tst_subset, batch_size=4, shuffle=False, collate_fn=collate_fn)

    # Initialize the model for this fold
    model = SimpleCNN(in_channels=917).to(device)
    auc, accuracy = train_and_evaluate(n_epochs, model, trn_loader, tst_loader)

In [ ]:
np.asarray(tile_grp)[tst_idx]

In [ ]:
plt.imshow(tile_ls[0][750,:,:], cmap='gray')